In [ ]:
spatial_reference = arcpy.SpatialReference(26915)
start_fc = "start.shp"
arcpy.management.CreateFeatureclass(arcpy.env.workspace, start_fc, "POINT", spatial_reference=spatial_reference)
with arcpy.da.InsertCursor(start_fc, ["SHAPE@"]) as cursor:
    point = arcpy.Point(568098.327959, 4886439.347936)
    point_geom = arcpy.PointGeometry(point)
    cursor.insertRow([point_geom])

finish_fc = "finish.shp"
arcpy.management.CreateFeatureclass(arcpy.env.workspace, finish_fc, "POINT", spatial_reference=spatial_reference)
with arcpy.da.InsertCursor(finish_fc, ["SHAPE@"]) as cursor:
    point = arcpy.Point(576511.759467, 4878355.428342)
    point_geom = arcpy.PointGeometry(point)
    cursor.insertRow([point_geom])

In [ ]:
import requests

base_url = "https://resources.gisdata.mn.gov/pub/data/elevation/lidar/q250k/q4342/laz/"
row = 28
column = 59
while row < 32:
    while column < 65:
        api_endpoint = f'4342-{row}-{column}.laz'
        url = f'{base_url}{api_endpoint}'
        local_filename = fr'C:\School\GIS 5571\Local\Lab2 Data\Part 2\laz\4342-{row}-{column}.laz'
        response = requests.get(url)
        column = column + 1
        with open(local_filename, 'wb') as f:
            f.write(response.content)
            
            
    row = row + 1
    column = 59

In [12]:
import arcpy

utm_sr = arcpy.SpatialReference(26915)
row = 28
column = 59
while row < 32:
    while column < 65:
        in_las = fr'C:\School\GIS 5571\Local\Lab2 Data\Part 2\laz\4342-{row}-{column}.laz'
        target_folder = r'C:\School\GIS 5571\Local\Lab2 Data\Part 2\las'
        arcpy.conversion.ConvertLas(in_las, target_folder, 
                                    out_las_dataset = fr'C:\School\GIS 5571\Local\Lab2 Data\Part 2\las\4342-{row}-{column}.lasd', 
                                    define_coordinate_system = 'ALL_FILES', 
                                    in_coordinate_system = utm_sr)
        column = column + 1
        
    row = row + 1
    column = 59

In [23]:
base_file_path = r'C:\School\GIS 5571\Local\Lab2 Data\Part 2\las'
out_las_dataset = r'C:\School\GIS 5571\Local\Lab2 Data\Part 2\merged.lasd'
arcpy.management.CreateLasDataset(base_file_path, 
                                  out_las_dataset, 
                                  spatial_reference = arcpy.SpatialReference(26915),
                                  create_las_prj = 'NO_FILES')

<Result 'C:\\School\\GIS 5571\\Local\\Lab2 Data\\Part 2\\merged.lasd'>

In [27]:
las_file = out_las_dataset
output_tin = r'C:\School\GIS 5571\Local\Lab2 Data\Part 2\Cost_Area_TIN'
arcpy.ddd.LasDatasetToTin(las_file, output_tin, "RANDOM", "PERCENT", 1) # Run the conversion tool

<Result 'C:\\School\\GIS 5571\\Local\\Lab2 Data\\Part 2\\Cost_Area_TIN'>

In [29]:
output_dem = r'C:\School\GIS 5571\Local\Lab2 Data\Part 2\DEM.tif'
arcpy.ddd.TinRaster(output_tin, output_dem, "FLOAT", "LINEAR",sample_distance="OBSERVATIONS 5000") # Run the conversion tool

<Result 'C:\\School\\GIS 5571\\Local\\Lab2 Data\\Part 2\\DEM.tif'>

In [33]:
arcpy.env.workspace = r'C:\School\GIS 5571\Local\Lab2 Data\Part 2'
output_slope = r'C:\School\GIS 5571\Local\Lab2 Data\Part 2\Slope.tif'
arcpy.ddd.Slope(output_dem, output_slope)

<Result 'C:\\School\\GIS 5571\\Local\\Lab2 Data\\Part 2\\Slope.tif'>

In [36]:
import zipfile

land_cover_resource = 'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/biota_landcover_nlcd_mn_2019/tif_biota_landcover_nlcd_mn_2019.zip'

final_response = requests.get(land_cover_resource)
file_path = r'C:\School\GIS 5571\Local\Lab2 Data\Part 2\landcover.zip'
with open(file_path, 'wb') as file:
    file.write(final_response.content)
        
extract_dir = r'C:\School\GIS 5571\Local\Lab2 Data\Part 2\Land Cover'
with zipfile.ZipFile(file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

In [38]:
coordinate_system = arcpy.SpatialReference(26915) 
in_workspace = r'C:\School\GIS 5571\Local\Lab2 Data\Part 2\Part 2\Default.gdb'
in_mosaicdataset_name = 'Study_Area'
arcpy.management.CreateMosaicDataset(in_workspace, in_mosaicdataset_name, coordinate_system)

in_mosaic_dataset = in_mosaicdataset_name
input_path = r'C:\School\GIS 5571\Local\Lab2 Data\Part 2\Slope.tif'
raster_type = 'Raster Dataset'
arcpy.management.AddRastersToMosaicDataset(in_mosaic_dataset, raster_type, input_path)

<Result 'Study_Area'>

In [40]:
arcpy.env.workspace = r'C:\School\GIS 5571\Local\Lab2 Data\Part 2\Part 2\Default.gdb'
mosaic_dataset = 'Study_Area\Footprint'  
output_shapefile = r'C:\School\GIS 5571\Local\Lab2 Data\Part 2\Footprint.shp'
arcpy.management.CopyFeatures(mosaic_dataset, output_shapefile)

<Result 'C:\\School\\GIS 5571\\Local\\Lab2 Data\\Part 2\\Footprint.shp'>

In [42]:
in_raster = r'C:\School\GIS 5571\Local\Lab2 Data\Part 2\Land Cover\NLCD_2019_Land_Cover.tif'
out_raster = r'C:\School\GIS 5571\Local\Lab2 Data\Part 2\Land_Cover_Clipped.tif'
rectangle = output_shapefile
arcpy.management.Clip(in_raster, rectangle, out_raster)

<Result 'C:\\School\\GIS 5571\\Local\\Lab2 Data\\Part 2\\Land_Cover_Clipped.tif'>

In [53]:
input_slope_raster = r'C:\School\GIS 5571\Local\Lab2 Data\Part 2\Slope.tif'
output_reclassified_raster = r'C:\School\GIS 5571\Local\Lab2 Data\Part 2\Slope_Reclass.tif'

reclassifications = "0 1.72 1;1.72 3.43 2;3.43 5.71 3;5.71 8.53 4;8.53 11.3 5;11.3 14.04 6;14.04 16.7 7;16.7 21.8 8;21.8 30.96 9;30.96 45 10;45 90 11"
arcpy.ddd.Reclassify(input_slope_raster, "VALUE", reclassifications, output_reclassified_raster, "NODATA")

<Result 'C:\\School\\GIS 5571\\Local\\Lab2 Data\\Part 2\\Slope_Reclass.tif'>

In [62]:
input_raster = r'C:\School\GIS 5571\Local\Lab2 Data\Part 2\Land_Cover_Clipped.tif'
output_raster = r'C:\School\GIS 5571\Local\Lab2 Data\Part 2\Land_Cover_Clipped_Reclass.tif'

reclassifications = "11 11 1;12 12 1;90 90 1;95 95 1;81 81 2;82 82 2"
arcpy.ddd.Reclassify(input_raster, "VALUE", reclassifications, output_raster, "NODATA")

<Result 'C:\\School\\GIS 5571\\Local\\Lab2 Data\\Part 2\\Land_Cover_Clipped_Reclass.tif'>

In [19]:
from arcpy.sa import *

arcpy.env.workspace = r'C:\School\GIS 5571\Local\Lab2 Data\Part 2'
slope_influence = 50
land_cover_influence = 50
farmland_cost = 6
water_crossing_cost = 11

env.workspace = r'C:\School\GIS 5571\Local\Lab2 Data\Part 2'
myWOTable = WOTable([["Slope_Reclass.tif", slope_influence,"VALUE",RemapValue([[1,11],[2,10],[3,9],
                    [4,8],[5,7],[6,6],[7,5],[8,4],[9,3],[10,2],[11,1],["NODATA","NODATA"]])],
                    ["Land_Cover_Clipped_Reclass.tif", land_cover_influence, "VALUE",
                     RemapValue([[1,water_crossing_cost],[2,farmland_cost],["NODATA",11]])]], [1,11,1])

outWeightedOverlay = WeightedOverlay(myWOTable)

In [20]:
input_points = r'C:\School\GIS 5571\Local\Lab2 Data\Part 2\start.shp'
outCostDist = CostDistance(input_points, outWeightedOverlay)

In [21]:
outBacklink = CostBackLink(input_points, outWeightedOverlay)

In [22]:
destination = r'C:\School\GIS 5571\Local\Lab2 Data\Part 2\finish.shp'
outCostPathWater = CostPath(destination, outCostDist, outBacklink)